In [1]:
import sys
sys.path.insert(0, '../gdaxtrader/')

In [2]:
from datetime import datetime, timedelta

In [3]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [4]:
import pandas as pd

In [9]:
import marketdata as mktdata
import visualize as vis

In [10]:
def toiso(unix):
    return datetime.fromtimestamp(unix).isoformat()

In [11]:
def iso_start_end(rates):
    return toiso(rates[len(rates)-1][0]), toiso(rates[0][0])

In [12]:
end = datetime.utcnow() - timedelta(weeks=1)
end2 = end - timedelta(days=200)
rates3, resp3 = mktdata.get_rates('BTC-USD', end_dt=end, sec_per_tick=60 * 60 * 24)
rates4, resp4 = mktdata.get_rates('BTC-USD', end_dt=end2, sec_per_tick=60 * 60 * 24)

2017-12-03 20:57:16 INFO     getting HISTORIC RATES
2017-12-03 20:57:16 INFO     performing GET request
2017-12-03 20:57:16 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 20:57:16 INFO     params:
2017-12-03 20:57:16 INFO     {'start': '2017-05-11T01:57:16.245132', 'end': '2017-11-27T01:57:16.245132', 'granularity': 86400}
2017-12-03 20:57:16 INFO     executing AUTHENTICATED request
2017-12-03 20:57:16 INFO     unix timestamp: 1512352636.2538528
2017-12-03 20:57:16 INFO     UTC ISO8601: 2017-12-04T01:57:16.253853
2017-12-03 20:57:21 INFO     GET request completed in 5.673916s.
2017-12-03 20:57:21 INFO     status: 200
2017-12-03 20:57:21 INFO     reason: OK
2017-12-03 20:57:21 INFO     getting HISTORIC RATES
2017-12-03 20:57:21 INFO     performing GET request
2017-12-03 20:57:21 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 20:57:21 INFO     params:
2017-12-03 20:57:21 INFO     {'start': '2016-10-23T01:57:16.245132', 'end': '2017-05-11T01

In [13]:
fig = vis.hist_rates(rates3 + rates4, product='BTC-USD', savetofile=False, movavg_windows=[14,28])

In [14]:
plotly.offline.iplot(fig)

In [ ]:
start_bal = 2000

In [ ]:
from __future__ import division

def mov_avg(rates, window=10):
    if len(rates) < window:
        return []
    
    mas = []
    for i, rate in enumerate(rates):
        if i < window - 1:
            continue
        
        mas_idx = i - window + 1
        
        # First moving average value: initialize it using the last 'window' values
        if len(mas) == 0:
            assert(mas_idx == 0)
            mas.append(sum(rates[:window]) / window)
            continue
        
        assert(mas_idx > 0)
        """
        The next moving average value is calculated as follows:
            
            mas_n    = ( (window - 1) * mas_{n-1} + rate ) / window
                    = mas_{n-1} * (window - 1) / window   +   rate / window
        """
        mas.append(mas[mas_idx-1] * ( (window-1) / window ) + (rate / window))
        
    return mas

In [ ]:
ratesdf = pd.DataFrame(rates3, columns=['unixTS', 'low', 'high', 'open', 'close', 'vol'])

In [ ]:
ratesdf.shape

In [ ]:
ma_close = mov_avg(ratesdf.close, window=14)
ma_open = mov_avg(ratesdf.open, window=14)
ma_low = mov_avg(ratesdf.low, window=14)
ma_high = mov_avg(ratesdf.high, window=14)